In [1]:
import numpy as np
import pandas as pd

df = pd.read_parquet("../Limpieza_Completada/cicids2017_CleanBinary.parquet")

ts_df = pd.read_parquet("../Timestamp_Datetime_Terminado/Timestamp_Tipo_Datetime.parquet")

print("df:", df.shape, "| ts_df:", ts_df.shape)



df: (2830539, 73) | ts_df: (2830539, 1)


In [ ]:
#Unión
df["Timestamp"] = ts_df["Timestamp"].values

print("Timestamp -> nulos:", df["Timestamp"].isna().sum(), "| dtype:", df["Timestamp"].dtype)



Timestamp -> nulos: 0 | dtype: datetime64[ns]


In [ ]:
# Ordenar
df = df.sort_values("Timestamp").reset_index(drop=True)

In [ ]:
#X e Y
y_raw = df["Label"].copy()  # o df["Attack"] si lo guardas, pero NO se usa en AE

X_df = df.select_dtypes(include=[np.number]).copy()



X_df: (2830539, 71) | y_raw: (2830539,)
¿Attack en X_df?: False


Split temporal (70 / 15 / 15)

In [ ]:
# Split temporal por índice
n = len(X_df)
i_train = int(n * 0.70)
i_val   = int(n * 0.85)

X_train_df = X_df.iloc[:i_train]
X_val_df   = X_df.iloc[i_train:i_val]
X_test_df  = X_df.iloc[i_val:]

print(X_train_df.shape, X_val_df.shape, X_test_df.shape)


(1981377, 71) (424581, 71) (424581, 71)


Escalado + PCA (fit SOLO en train)

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train_df)
X_val_s   = scaler.transform(X_val_df)
X_test_s  = scaler.transform(X_test_df)

pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_s)
X_val_pca   = pca.transform(X_val_s)
X_test_pca  = pca.transform(X_test_s)

print("Componentes PCA:", X_train_pca.shape[1])


Componentes PCA: 26


Ventanas Temporales(Solo Train)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

WINDOW_SIZE = 20
STRIDE = 5

class WindowDataset(Dataset):
    def __init__(self, X, window_size, stride):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.window_size = window_size
        self.stride = stride
        self.indices = list(range(0, len(X) - window_size + 1, stride))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        i = self.indices[idx]
        window = self.X[i:i+self.window_size]
        return window, window  #input = target (autoencoder)


In [9]:
train_dataset = WindowDataset(X_train_pca, WINDOW_SIZE, STRIDE)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)

print("Nº ventanas train:", len(train_dataset))


Nº ventanas train: 396272


In [ ]:
#Val windows sin shuffle
val_dataset = WindowDataset(X_val_pca, WINDOW_SIZE, STRIDE)
val_loader  = DataLoader(val_dataset, batch_size=256, shuffle=False, drop_last=False)

print("Ventanas val:", len(val_dataset))


Ventanas val: 84913


Autoencoder GRU 

In [ ]:
import torch
import torch.nn as nn

device = torch.device("cpu")

class GRUAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.GRU(hidden_dim, input_dim, batch_first=True)

    def forward(self, x):
        # x: (B, T, D)
        _, h = self.encoder(x)  # h: (1, B, H)

        # Repetimos el estado oculto para reconstruir T pasos
        T = x.size(1)
        h_rep = h.repeat(T, 1, 1).permute(1, 0, 2)  # (B, T, H)

        out, _ = self.decoder(h_rep)  # (B, T, D)
        return out

input_dim = X_train_pca.shape[1]
hidden_dim = 64

model = GRUAutoencoder(input_dim, hidden_dim).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

print("Modelo listo en:", device)


Modelo listo en: cpu


In [ ]:
import copy
import numpy as np
import torch

EPOCHS = 20    
PATIENCE = 3 

best_val = float("inf")
best_state = None
pat = 0

for epoch in range(EPOCHS):
    # ---- TRAIN ----
    model.train()
    train_loss = 0.0

    for xb, yb in train_loader:
        xb = xb.to(device) 
        yb = yb.to(device)

        noise = torch.randn_like(xb) * 0.05 #Ruido gaussiano suave
        xb_noisy = xb + noise

        optimizer.zero_grad()
        recon = model(xb_noisy)
        loss = criterion(recon, yb)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # ---- VAL ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            yb = yb.to(device)

            #Sin ruido en Val
            recon = model(xb)
            loss = criterion(recon, yb)
            val_loss += loss.item()

    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS} | train={train_loss:.6f} | val={val_loss:.6f}")

    # ---- CHECKPOINT ----
    if val_loss < best_val - 1e-5:
        best_val = val_loss
        best_state = copy.deepcopy(model.state_dict())
        pat = 0
        print("  ✓ Nuevo mejor modelo")
    else:
        pat += 1
        print(f"  · No mejora (patience {pat}/{PATIENCE})")

    # ---- EARLY STOPPING ----
    if pat >= PATIENCE:
        print("Early stopping activado.")
        break

# Cargar mejor modelo y guardar
model.load_state_dict(best_state)
torch.save(model.state_dict(), "gru_autoencoder_pca_trainonly_best.pt")
print("Guardado: gru_autoencoder_pca_trainonly_best.pt | best_val:", best_val)


Epoch 1/20 | train=2.297568 | val=0.856266
  ✓ Nuevo mejor modelo
Epoch 2/20 | train=2.267718 | val=0.823064
  ✓ Nuevo mejor modelo
Epoch 3/20 | train=2.242814 | val=0.812399
  ✓ Nuevo mejor modelo
Epoch 4/20 | train=2.230907 | val=0.805524
  ✓ Nuevo mejor modelo
Epoch 5/20 | train=2.223228 | val=0.801729
  ✓ Nuevo mejor modelo
Epoch 6/20 | train=2.218001 | val=0.799302
  ✓ Nuevo mejor modelo
Epoch 7/20 | train=2.213951 | val=0.797667
  ✓ Nuevo mejor modelo
Epoch 8/20 | train=2.210061 | val=0.796529
  ✓ Nuevo mejor modelo
Epoch 9/20 | train=2.207198 | val=0.794897
  ✓ Nuevo mejor modelo
Epoch 10/20 | train=2.204052 | val=0.793769
  ✓ Nuevo mejor modelo
Epoch 11/20 | train=2.200531 | val=0.792445
  ✓ Nuevo mejor modelo
Epoch 12/20 | train=2.199550 | val=0.791996
  ✓ Nuevo mejor modelo
Epoch 13/20 | train=2.197381 | val=0.791198
  ✓ Nuevo mejor modelo
Epoch 14/20 | train=2.196023 | val=0.790762
  ✓ Nuevo mejor modelo
Epoch 15/20 | train=2.193235 | val=0.790261
  ✓ Nuevo mejor modelo
Epoc

Guardado Scaler y PCA


In [13]:
import joblib

joblib.dump(scaler, "scaler_trainonly.joblib")
joblib.dump(pca, "pca_trainonly.joblib")

print("Guardado scaler_trainonly.joblib y pca_trainonly.joblib")


Guardado scaler_trainonly.joblib y pca_trainonly.joblib


Cargar Modelo para evitar tiempo en ejecutar Gru


In [ ]:
import torch

device = torch.device("cpu")

model = GRUAutoencoder(input_dim=X_train_pca.shape[1], hidden_dim=64).to(device)
model.load_state_dict(torch.load("gru_autoencoder_pca_trainonly_best.pt", map_location=device))
model.eval()


Modelo cargado desde gru_autoencoder_pca_trainonly_best.pt
